- 学修番号: 21445095
- 氏名: 小田龍司

# 課題 2.1　決定木分析

- 2.1.1 AUCが最大になる max_depth
- 2.1.2 決定木の描画
- 2.1.3 散布図の描画

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, roc_curve, auc
from sklearn.tree import plot_tree

autos = pd.read_csv('data/imports-85.data', na_values='?')
autos.columns = ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration',
                    'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
                    'wheel_base', 'length','width', 'height', 'curb_weight',
                    'engine_type', 'num_of_cylinders', 'engine_size', 'fuel_system',
                    'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm',
                    'city_mpg', 'highway_mpg','price']
# 表示する最大列数の設定
pd.set_option('display.max_columns', len(autos.columns))

### 2.1 で用いるDataFrame

autos から symboling, normalized_losses, city_mpg, highway_mpg 列を除外したうえで、highway_mpgから中央値よりも大きければ1、小さいか等しければ0を値としたhighway_mpg_01列を追加した pandas DataFrame を作成の後、欠損値 NaN がある行を除去。

In [ ]:
df = autos.drop(['symboling', 'normalized_losses', 'city_mpg', 'highway_mpg'], axis=1)
# ダミー変数化
highway_mpg_median = autos.highway_mpg.median()
df['highway_mpg_01'] = \
    autos['highway_mpg'].map(lambda x: 1 if x > highway_mpg_median else 0)
df = pd.get_dummies(data=df, drop_first=True)
# 欠損値 NaN がある行の除去
df = df.dropna()
# 確認
print(df.groupby('highway_mpg_01').size())
df.head(1)

<hr>

highway_mpg_01 を目的変数、それ以外を説明変数とし、決定木を用いて以下に答えなさい。
- 分割の規準は、エントロピー (criterion='entropy') を用いること。

### 2.1.1 AUCが最大になる max_depth

交差検証により、AUCが最大になる max_depth を求めなさい。
- 分割数は 10 とする。

<hr>

### 2.1.2 決定木の描画

2.1.1で求めた max_depth の決定木を描画しなさい。
- class_names は適切に付けること。

<hr>

### 2.1.3 散布図の描画

2.1.2で作成した決定木における正規化された情報利得の総和の上位2つとなる説明変数の散布図を描画しなさい。
- highway_mpg_01の値で色分けすること。
- 2.1.2の説明変数のうち大きい方をX軸とすること。
- 2.1.2で作成した決定木の情報から領域を分割する直線を重ねてひくこと。